In [1]:
import shutil
import os
from pathlib import Path

# Define source and destination directories
src_dir = Path('/kaggle/input/visdrone-det-img/VisDrone')
dst_dir = Path('/kaggle/working/VisDrone')

# Create destination directory if it doesn't exist
dst_dir.mkdir(parents=True, exist_ok=True)

# Copy dataset to writable directory
for item in src_dir.iterdir():
    s = item
    d = dst_dir / item.name
    if item.is_dir():
        shutil.copytree(s, d, symlinks=True, ignore=shutil.ignore_patterns('*.zip'))
    else:
        shutil.copy2(s, d)


In [2]:
!mv '/kaggle/working/VisDrone/VisDrone2019-DET-train (1)' /kaggle/working/VisDrone/VisDrone2019-DET-train

In [3]:
import os
from pathlib import Path
from PIL import Image
from tqdm import tqdm
import yaml

# Define YAML content
content = {
    'path': '/kaggle/working/VisDrone',  # dataset root dir
    'train': '/kaggle/working/VisDrone/VisDrone2019-DET-train/VisDrone2019-DET-train/',  # train images
    'val': '/kaggle/working/VisDrone/VisDrone2019-DET-val/VisDrone2019-DET-val/',  # val images
    'test': '/kaggle/working/VisDrone/VisDrone2019-DET-test-dev/VisDrone2019-DET-test-dev/',  # test images
    'names': {
        0: 'pedestrian',
        1: 'people',
        2: 'bicycle',
        3: 'car',
        4: 'van',
        5: 'truck',
        6: 'tricycle',
        7: 'awning-tricycle',
        8: 'bus',
        9: 'motor'
    }
}

# # Define the path to save the YAML file
# yaml_file_path = '/kaggle/working/dataset.yaml'

# # Create the directory if it does not exist
# yaml_dir = os.path.dirname(yaml_file_path)
# if not os.path.exists(yaml_dir):
#     os.makedirs(yaml_dir)

# # Write the YAML content to the file
# with open(yaml_file_path, 'w') as file:
#     yaml.dump(content, file, default_flow_style=False)

# print(f'YAML file created at: {yaml_file_path}')

# Function to convert VisDrone annotations to YOLO format
def visdrone2yolo(dir):
    def convert_box(size, box):
        # Convert VisDrone box to YOLO xywh box
        dw = 1. / size[0]
        dh = 1. / size[1]
        return (box[0] + box[2] / 2) * dw, (box[1] + box[3] / 2) * dh, box[2] * dw, box[3] * dh

    (dir / 'labels').mkdir(parents=True, exist_ok=True)  # Make labels directory
    pbar = tqdm((dir / 'annotations').glob('*.txt'), desc=f'Converting {dir}')
    for f in pbar:
        img_size = Image.open((dir / 'images' / f.stem).with_suffix('.jpg')).size
        lines = []
        with open(f, 'r') as file:  # Read annotation.txt
            for row in [x.split(',') for x in file.read().strip().splitlines()]:
                if row[4] == '0':  # VisDrone 'ignored regions' class 0
                    continue
                cls = int(row[5]) - 1
                box = convert_box(img_size, tuple(map(int, row[:4])))
                lines.append(f"{cls} {' '.join(f'{x:.6f}' for x in box)}\n")
        # Save YOLO formatted labels
        label_path = str(f).replace(f'{os.sep}annotations{os.sep}', f'{os.sep}labels{os.sep}')
        with open(label_path, 'w') as fl:
            fl.writelines(lines)

# Convert annotations for the datasets
root_dir = Path('/kaggle/working/VisDrone')
for d in ['VisDrone2019-DET-train/VisDrone2019-DET-train', 'VisDrone2019-DET-val/VisDrone2019-DET-val', 'VisDrone2019-DET-test-dev/VisDrone2019-DET-test-dev']:
    visdrone2yolo(root_dir / d)  # Convert VisDrone annotations to YOLO labels

print("Conversion complete.")


Converting /kaggle/working/VisDrone/VisDrone2019-DET-train/VisDrone2019-DET-train: 6471it [00:04, 1383.87it/s]
Converting /kaggle/working/VisDrone/VisDrone2019-DET-val/VisDrone2019-DET-val: 548it [00:00, 1214.94it/s]
Converting /kaggle/working/VisDrone/VisDrone2019-DET-test-dev/VisDrone2019-DET-test-dev: 0it [00:00, ?it/s]

Conversion complete.


In [4]:
!rm -r /kaggle/working/VisDrone/VisDrone2019-DET-train/VisDrone2019-DET-train/annotations
!rm -r /kaggle/working/VisDrone/VisDrone2019-DET-val/VisDrone2019-DET-val/annotations


In [5]:
!rm -r /kaggle/working/VisDrone/VisDrone2019-DET-test-dev
!rm -r /kaggle/working/VisDrone/VisDrone2019-DET-test-challenge

In [6]:
import yaml

# Define YAML content
content = {
    'path': '/kaggle/working/VisDrone',  # dataset root dir
    'train': '/kaggle/working/VisDrone/VisDrone2019-DET-train/VisDrone2019-DET-train/',  # train images
    'val': '/kaggle/working/VisDrone/VisDrone2019-DET-val/VisDrone2019-DET-val/',  # val images
    'names': {
        0: 'pedestrian',
        1: 'people',
        2: 'bicycle',
        3: 'car',
        4: 'van',
        5: 'truck',
        6: 'tricycle',
        7: 'awning-tricycle',
        8: 'bus',
        9: 'motor'
    }
}

# Define the path to save the YAML file
yaml_file_path = '/kaggle/working/VisDrone.yaml'

# Create the directory if it does not exist
yaml_dir = os.path.dirname(yaml_file_path)
if not os.path.exists(yaml_dir):
    os.makedirs(yaml_dir)

# Write the YAML content to the file
with open(yaml_file_path, 'w') as file:
    yaml.dump(content, file, default_flow_style=False)

print(f'YAML file created at: {yaml_file_path}')


YAML file created at: /kaggle/working/VisDrone.yaml


In [7]:
pip install ultralytics


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 863.7/863.7 kB 22.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install -U ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 54.4 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.7
    Uninstalling widgetsnbextension-3.6.7:
      Successfully uninstalled widgetsnbextension-3.6.7
  Attempting uninstall: jupyterlab-widgets
    Found existing installation: jupyterlab-widgets 3.0.9
    Uninstalling jupyterlab-widgets-3.0.9:
      Successfully uninstalled jupyterlab-widgets-3.0.9
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip uninstall -y wandb

Found existing installation: wandb 0.17.4
Uninstalling wandb-0.17.4:
  Successfully uninstalled wandb-0.17.4
Note: you may need to restart the kernel to use updated packages.


In [10]:
import ultralytics
from ultralytics import YOLO

model=YOLO('yolov8x.pt')
results=model.train(data='/kaggle/working/VisDrone.yaml',epochs=30 ,imgsz=640, cos_lr=True, amp=False, optimizer='Adam',hsv_h=0.2,perspective=0.0005,auto_augment='augmix',batch=8) 


100%|██████████| 131M/131M [00:00<00:00, 280MB/s]


Ultralytics YOLOv8.2.71 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=/kaggle/working/VisDrone.yaml, epochs=30, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=False, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=Tru

100%|██████████| 755k/755k [00:00<00:00, 22.0MB/s]


Overriding model.yaml nc=80 with nc=10

                   from  n    params  module                                       arguments                     
  0                  -1  1      2320  ultralytics.nn.modules.conv.Conv             [3, 80, 3, 2]                 
  1                  -1  1    115520  ultralytics.nn.modules.conv.Conv             [80, 160, 3, 2]               
  2                  -1  3    436800  ultralytics.nn.modules.block.C2f             [160, 160, 3, True]           
  3                  -1  1    461440  ultralytics.nn.modules.conv.Conv             [160, 320, 3, 2]              
  4                  -1  6   3281920  ultralytics.nn.modules.block.C2f             [320, 320, 6, True]           
  5                  -1  1   1844480  ultralytics.nn.modules.conv.Conv             [320, 640, 3, 2]              
  6                  -1  6  13117440  ultralytics.nn.modules.block.C2f             [640, 640, 6, True]           
  7                  -1  1   3687680  ultralytic

train: Scanning /kaggle/working/VisDrone/VisDrone2019-DET-train/VisDrone2019-DET-train/labels... 6471 images, 0 backgrounds, 0 corrupt: 100%|██████████| 6471/6471 [00:05<00:00, 1096.59it/s]

...

                   all        548      38759      0.495      0.388      0.399      0.239

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      30/30      14.5G       1.23     0.8406     0.9372        208        640: 100%|██████████| 809/809 [12:33<00:00,  1.07it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 35/35 [00:11<00:00,  3.03it/s]


                   all        548      38759      0.494      0.398      0.406      0.243

30 epochs completed in 6.436 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 136.7MB
Optimizer stripped from runs/detect/train/weights/best.pt, 136.7MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.71 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Model summary (fused): 268 layers, 68,133,198 parameters, 0 gradients, 257.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 35/35 [00:32<00:00,  1.07it/s]


                   all        548      38759      0.494      0.398      0.406      0.243
            pedestrian        520       8844      0.529      0.412      0.437      0.197
                people        482       5125      0.596      0.261      0.336      0.125
               bicycle        364       1287      0.186      0.172      0.111     0.0475
                   car        515      14064      0.742      0.775      0.805      0.572
                   van        421       1975      0.496      0.485      0.475      0.337
                 truck        266        750      0.433      0.409      0.385       0.26
              tricycle        337       1045      0.467      0.298        0.3      0.169
       awning-tricycle        220        532      0.272      0.199      0.173      0.106
                   bus        131        251      0.648      0.554      0.595      0.424
                 motor        485       4886      0.569      0.413      0.445      0.195
Speed: 0.1ms preproce